In [1]:
!pip install ipdb
!pip install transformers

# Package loading 

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import numpy as np

import ipdb
import spacy
spacy.load("en_core_web_sm")
import torch
from torchtext import data
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

from sklearn.model_selection import KFold, StratifiedKFold

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Data Exploration

In [3]:
train = pd.read_csv("./data/Train_process.csv")
test = pd.read_csv("./data/Test_process.csv")

# train = pd.read_csv("./data/Train.csv")
# test = pd.read_csv("./data/Test.csv")

sample = pd.read_csv("./data/SampleSubmission.csv")

y_kfold = train.label.values # this is used down in the the kfold

In [4]:
train.tail()

,ID,text,label
611,BOHSNXCN,what should i do to stop alcoholism ?,Alcohol
612,GVDXRQPY,how to become my oneself again,Suicide
613,IO4JHIQS,how can someone stop it ?,Alcohol
614,1DS3P1XO,i feel unworthy,Depression
615,ORF71PVQ,i feel so discouraged with life,Depression


Let check the distribution of the data. 

In [5]:
test.tail()

,ID,text
304,Z9A6ACLK,yes
305,ZDUOIGKN,my girlfriend dumped me
306,ZHQ60CCH,how can i go back to being my old self ?
307,ZVIJMA4O,is it true hang is medicinal ?
308,ZYIFAY98,how can i overcome the problem ?


In [6]:
sample.tail()

,ID,Depression,Alcohol,Suicide,Drugs
304,Z9A6ACLK,0,0,0,0
305,ZDUOIGKN,0,0,0,0
306,ZHQ60CCH,0,0,0,0
307,ZVIJMA4O,0,0,0,0
308,ZYIFAY98,0,0,0,0


In [7]:
train.label.value_counts()

Depression    352
Alcohol       140
Suicide        66
Drugs          58
Name: label, dtype: int64

In [8]:
list(np.unique(train.label))

['Alcohol', 'Depression', 'Drugs', 'Suicide']

In [9]:
# # This plot need to be checked 
# plt.pie(list(train.len.value_counts()), labels=list(np.unique(train.len)))
# plt.show()

## Preparing Data

One of the main concepts of TorchText is the `Field`. These define how your data should be processed. In our sentiment classification task the data consists of both the raw string of the review and the sentiment, either "pos" or "neg".

The parameters of a `Field` specify how the data should be processed. 

We use the `TEXT` field to define how the review should be processed, and the `LABEL` field to process the sentiment. 

Our `TEXT` field has `tokenize='spacy'` as an argument. This defines that the "tokenization" (the act of splitting the string into discrete "tokens") should be done using the [spaCy](https://spacy.io) tokenizer. If no `tokenize` argument is passed, the default is simply splitting the string on spaces.

`LABEL` is defined by a `LabelField`, a special subset of the `Field` class specifically used for handling labels. We will explain the `dtype` argument later.

For more on `Fields`, go [here](https://github.com/pytorch/text/blob/master/torchtext/data/field.py).

We also set the random seeds for reproducibility.

In [10]:
SEED = 1234

# torch.manual_seed(SEED)
# torch.backends.cudnn.deterministic = True

# # TEXT = data.Field(tokenize = 'spacy')
# LABELS = data.LabelField(dtype = torch.float)

# TEXT = data.Field(batch_first = True,
#                   use_vocab = False,
# #                   tokenize = tokenize_and_cut,
#                   preprocessing = tokenizer.convert_tokens_to_ids,
#                   init_token = init_token_idx,
#                   eos_token = eos_token_idx,
#                   pad_token = pad_token_idx,
#                   unk_token = unk_token_idx)

import torch
from torchtext import data
from torchtext import datasets
import random

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy')
LABELS = data.LabelField(dtype = torch.float)

In [11]:
train_data, test_data = data.TabularDataset.splits(
    path='./data', train='Train_process.csv',
    test='Test_process.csv', format='csv', skip_header=True,
    fields=[('ID', None),
            ('text', TEXT),
            ('label', LABELS)])


In [12]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 616
Number of testing examples: 309


Let check one example

In [13]:
print(vars(train_data.examples[0]))

{'text': ['i', 'feel', 'that', 'it', 'was', 'better', 'i', 'die', 'am', 'happy'], 'label': 'Depression'}


In [14]:
vars(train_data[-1])

{'label': 'Depression',
 'text': ['i', 'feel', 'so', 'discouraged', 'with', 'life']}

In [15]:
# MAX_VOCAB_SIZE = 25_000

# glove.6B.300d": 862 M

TEXT.build_vocab(train_data, 
#                  max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.300d",  #glove.twitter.27B.200d
                 unk_init = torch.Tensor.normal_)

LABELS.build_vocab(train_data)

## Split 

In [16]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED), 
                                          stratified=True, split_ratio=0.8)

In [17]:
print(f'Number of training examples: {len(train_data)}')
# print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 493
Number of testing examples: 309


In [18]:
# print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABELS.vocab)}")

Unique tokens in LABEL vocabulary: 4


In [19]:
print(LABELS.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7f9a5cf4e2f0>, {'Depression': 0, 'Alcohol': 1, 'Suicide': 2, 'Drugs': 3})


In [20]:
print(vars(valid_data.examples[-1]))

{'text': ['what', 'are', 'the', 'effects', 'of', 'depression', '?'], 'label': 'Depression'}


The final step of preparing the data is creating the iterators. We iterate over these in the training/evaluation loop, and they return a batch of examples (indexed and converted into tensors) at each iteration.

We'll use a BucketIterator which is a special type of iterator that will return a batch of examples where each example is of a similar length, minimizing the amount of padding per example.

We also want to place the tensors returned by the iterator on the GPU (if you're using one). PyTorch handles this using torch.device, we then pass this device to the iterator.

In [21]:
BATCH_SIZE = 32

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort=False,
    device = device)

# train_iterator, test_iterator = data.BucketIterator.splits(
#     (train_data, test_data), 
#     batch_size = BATCH_SIZE,
#     sort=False,
#     device = device)

In [22]:
# next(test_iterator.__iter__())

# Build the Model

In [23]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, text):
        
        #text = [sent len, batch size]
        
        text = text.permute(1, 0)
                
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
            
        #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))

        #cat = [batch size, n_filters * len(filter_sizes)]
        
        out = self.fc(cat)
        
#         out = self.sigmoid(out)
            
        return out #self.fc(cat) #

In [24]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300 #100
N_FILTERS = 100
FILTER_SIZES = [2,3,4]
OUTPUT_DIM = len(LABELS.vocab)
DROPOUT = 0.3
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

In [25]:
# # can view the summary if you like
# model.to(device)
# summary(model, (16, 32))

In [26]:
# MAX_VOCAB_SIZE = 25_000

# TEXT.build_vocab(train_data, 
#                  max_size = MAX_VOCAB_SIZE, 
#                  vectors = "glove.6B.100d", 
#                  unk_init = torch.Tensor.normal_)

# LABEL.build_vocab(train_data)

Let's also create a function that will tell us how many trainable parameters our model has so we can compare the number of parameters across different models.

In [27]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 512,104 trainable parameters


In [28]:
# for name, param in model.named_parameters():                
#     if name.startswith('bert'):
#         param.requires_grad = False

In [29]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 512,104 trainable parameters


In [30]:
for name, param in model.named_parameters():                
    if param.requires_grad:
        print(name)

embedding.weight
convs.0.weight
convs.0.bias
convs.1.weight
convs.1.bias
convs.2.weight
convs.2.bias
fc.weight
fc.bias


# Train the model

Next, we'll load our pre-trained embeddings.

In [31]:
pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ..., -1.4447,  0.8402, -0.8668],
        [ 0.1032, -1.6268,  0.5729,  ...,  0.3180, -0.1626, -0.0417],
        [-0.1329,  0.1699, -0.1436,  ..., -0.2378,  0.1477,  0.6290],
        ...,
        [ 0.5390,  0.0090,  0.2787,  ..., -0.1150, -0.4234,  0.0318],
        [-0.3364, -0.1851, -0.1022,  ..., -0.1311, -0.0346,  0.3446],
        [-0.2117,  0.1462,  0.0605,  ..., -0.4895, -0.0707, -0.0814]])

Then zero the initial weights of the unknown and padding tokens.

In [32]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

Another different to the previous notebooks is our loss function (aka criterion). Before we used `BCEWithLogitsLoss`, however now we use `CrossEntropyLoss`. Without going into too much detail, `CrossEntropyLoss` performs a *softmax* function over our model outputs and the loss is given by the *cross entropy* between that and the label.

Generally:
- `CrossEntropyLoss` is used when our examples exclusively belong to one of $C$ classes
- `BCEWithLogitsLoss` is used when our examples exclusively belong to only 2 classes (0 and 1) and is also used in the case where our examples belong to between 0 and $C$ classes (aka multilabel classification).

In [33]:
from collections import Counter
class_distrbution = Counter()
for text in train["label"].values:
    class_distrbution[text] += 1
        
class_distrbution.most_common(10)

[('Depression', 352), ('Alcohol', 140), ('Suicide', 66), ('Drugs', 58)]

In [34]:
np.sum(list(class_distrbution.values()))

616

In [35]:
class_distrbution

Counter({'Alcohol': 140, 'Depression': 352, 'Drugs': 58, 'Suicide': 66})

In [36]:
LABELS.vocab.stoi.keys()

dict_keys(['Depression', 'Alcohol', 'Suicide', 'Drugs'])

In [37]:
# criterion = nn.CrossEntropyLoss(weight = x)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# train_loader_origin.dataset.classes,
class_weights = [class_distrbution[i] for i in LABELS.vocab.stoi.keys()]
class_weights_normalized = [max(class_weights)/i for i in class_weights]

class_weights_normalized ,torch.Tensor(class_weights_normalized)

w = torch.Tensor(class_weights_normalized)
w = w.to(device)
# x = x
w,class_distrbution

(tensor([1.0000, 2.5143, 5.3333, 6.0690], device='cuda:0'),
 Counter({'Alcohol': 140, 'Depression': 352, 'Drugs': 58, 'Suicide': 66}))

In [38]:
class_weights, w

([352, 140, 66, 58], tensor([1.0000, 2.5143, 5.3333, 6.0690], device='cuda:0'))

In [39]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

criterion = nn.CrossEntropyLoss()
# criterion = nn.CrossEntropyLoss(weight=w)



model = model.to(device)
criterion = criterion.to(device)

Before, we had a function that calculated accuracy in the binary label case, where we said if the value was over 0.5 then we would assume it is positive. In the case where we have more than 2 classes, our model outputs a $C$ dimensional vector, where the value of each element is the beleief that the example belongs to that class. 

For example, in our labels we have: 'HUM' = 0, 'ENTY' = 1, 'DESC' = 2, 'NUM' = 3, 'LOC' = 4 and 'ABBR' = 5. If the output of our model was something like: **[5.1, 0.3, 0.1, 2.1, 0.2, 0.6]** this means that the model strongly believes the example belongs to class 0, a question about a human, and slightly believes the example belongs to class 3, a numerical question.

We calculate the accuracy by performing an `argmax` to get the index of the maximum value in the prediction for each element in the batch, and then counting how many times this equals the actual label. We then average this across the batch.

In [40]:
from sklearn.metrics import accuracy_score, precision_score

def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    
    correct = max_preds.squeeze(1).eq(y)
#     ipdb.set_trace()
    pr = correct.sum().item() / torch.FloatTensor([y.shape[0]])
    
#     pr = precision_score(y.detach().numpy(), 
#                            max_preds.argmax(dim = 1, 
#                                         keepdim = True).detach().numpy(), average='macro')
#     ipdb.set_trace()
    return pr

The training loop is similar to before, without the need to `squeeze` the model predictions as `CrossEntropyLoss` expects the input to be **[batch size, n classes]** and the label to be **[batch size]**.

The label needs to be a `LongTensor`, which it is by default as we did not set the `dtype` to a `FloatTensor` as before.

In [41]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        
        predictions = model(batch.text)
#         ipdb.set_trace()
        
        loss = criterion(predictions, batch.label.long())
        
        acc = categorical_accuracy(predictions, batch.label.long())
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

The evaluation loop is, again, similar to before.

In [42]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text)
            
            loss = criterion(predictions, batch.label.long())
            
            acc = categorical_accuracy(predictions, batch.label.long())

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [43]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Next, we train our model.

In [44]:
N_EPOCHS = 10

best_valid_loss = 0.28 #float('inf')
best_valid_acc = 0.88

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        print('Saving Model ...')
        torch.save(model.state_dict(), 'Best_Model_Bert_'+str(best_valid_loss)[:4]+'.pt')
        print('*****************************************************')
        print('best record: [epoch %d], [val loss %.5f], [val acc %.5f]' % (epoch, valid_loss, valid_acc))
        print('*****************************************************')

Epoch: 01 | Epoch Time: 0m 0s
	Train Loss: 0.904 | Train Acc: 67.11%
	 Val. Loss: 0.643 |  Val. Acc: 77.98%
Epoch: 02 | Epoch Time: 0m 0s
	Train Loss: 0.468 | Train Acc: 84.69%
	 Val. Loss: 0.466 |  Val. Acc: 83.45%
Epoch: 03 | Epoch Time: 0m 0s
	Train Loss: 0.289 | Train Acc: 91.32%
	 Val. Loss: 0.409 |  Val. Acc: 85.01%
Epoch: 04 | Epoch Time: 0m 0s
	Train Loss: 0.202 | Train Acc: 94.46%
	 Val. Loss: 0.399 |  Val. Acc: 84.23%
Epoch: 05 | Epoch Time: 0m 0s
	Train Loss: 0.138 | Train Acc: 96.78%
	 Val. Loss: 0.399 |  Val. Acc: 84.23%
Epoch: 06 | Epoch Time: 0m 0s
	Train Loss: 0.088 | Train Acc: 99.02%
	 Val. Loss: 0.395 |  Val. Acc: 85.79%
Epoch: 07 | Epoch Time: 0m 0s
	Train Loss: 0.062 | Train Acc: 99.41%
	 Val. Loss: 0.446 |  Val. Acc: 85.01%
Epoch: 08 | Epoch Time: 0m 0s
	Train Loss: 0.048 | Train Acc: 99.41%
	 Val. Loss: 0.413 |  Val. Acc: 84.87%
Epoch: 09 | Epoch Time: 0m 0s
	Train Loss: 0.034 | Train Acc: 99.61%
	 Val. Loss: 0.424 |  Val. Acc: 84.09%
Epoch: 10 | Epoch Time: 0m 0

# Cross validation

In [9]:
from sklearn.metrics import accuracy_score, precision_score

def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    
    correct = max_preds.squeeze(1).eq(y)
#     ipdb.set_trace()
    pr = correct.sum().item() / torch.FloatTensor([y.shape[0]])
    
#     pr = precision_score(y.detach().numpy(), 
#                            max_preds.argmax(dim = 1, 
#                                         keepdim = True).detach().numpy(), average='macro')
#     ipdb.set_trace()
    return pr

def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        
        predictions = model(batch.text)
#         ipdb.set_trace()
        
        loss = criterion(predictions, batch.label.long())
        
        acc = categorical_accuracy(predictions, batch.label.long())
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text)
            
            loss = criterion(predictions, batch.label.long())
            
            acc = categorical_accuracy(predictions, batch.label.long())

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [10]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, text):
        
        #text = [sent len, batch size]
        
        text = text.permute(1, 0)
                
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
            
        #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))

        #cat = [batch size, n_filters * len(filter_sizes)]
        
        out = self.fc(cat)
        
#         out = self.sigmoid(out)
            
        return out #self.fc(cat) #

In [11]:
class load_data(object):
    def __init__(self, SEED=1234):
        torch.manual_seed(SEED)
        torch.cuda.manual_seed(SEED)
        torch.backends.cudnn.deterministic = True

        TEXT = data.Field(tokenize = 'spacy')
        LABELS = data.LabelField(dtype = torch.float)


#         self.train_data, self.test_data = datasets.IMDB.splits(TEXT, LABEL)
        
        self.train_data, self.test_data = data.TabularDataset.splits(
            path='./data', train='Train_process.csv',
            test='Test_process.csv', format='csv', skip_header=True,
            fields=[('ID', None),
                    ('text', TEXT),
                    ('label', LABELS)])

        self.SEED = SEED


    def get_fold_data(self, num_folds=10):
        """
        More details about 'fields' are available at 
        https://github.com/pytorch/text/blob/master/torchtext/datasets/imdb.py
        """

        TEXT = data.Field(tokenize='spacy')
        LABELS = data.LabelField(dtype=torch.float)
        fields = [('text', TEXT), ('label', LABELS)]
        
#         kf = KFold(n_splits=num_folds, shuffle=True, random_state=self.SEED)
        kf = StratifiedKFold(n_splits=num_folds, random_state=self.SEED)
        
        train_data_arr = np.array(self.train_data.examples)

#         ipdb.set_trace()
        for train_index, val_index in kf.split(train_data_arr, y_kfold):
            yield(
                TEXT,
                LABELS,
                data.Dataset(train_data_arr[train_index], fields=fields),
                data.Dataset(train_data_arr[val_index], fields=fields),
            )
    
    def get_test_data(self):
        return self.test_data

In [12]:
# KFold?

In [13]:
data_generator = load_data()
_history = []
device = None
model = None
criterion = None
fold_index = 0
num_folds = 5
batch_size = 32
epochs = 5

import torch.optim as optim

# optimizer = optim.Adam(model.parameters())

best_valid_acc = 0.9
best_val_loss = 0.25
    
for TEXT, LABELS, train_data, val_data in data_generator.get_fold_data(num_folds=num_folds):
    print("***** Running Training *****")
    print(f"Now fold: {fold_index + 1} / {num_folds}")

    TEXT.build_vocab(train_data,
                     vectors = "glove.6B.300d",
                     unk_init = torch.Tensor.normal_)
    
    print(f'Embedding size: {TEXT.vocab.vectors.size()}.')
    LABELS.build_vocab(train_data) # For converting str into float labels.
    
#     Model(len(TEXT.vocab), embedding_dim, hidden_dim,
#         output_dim, num_layers, dropout, TEXT.vocab.vectors, embedding_trainable)
    
    INPUT_DIM = len(TEXT.vocab)
    EMBEDDING_DIM = 300
    N_FILTERS = 100
    FILTER_SIZES = [2,3,4]
    OUTPUT_DIM = len(LABELS.vocab)
    DROPOUT = 0.308
    PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
    
    model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

    pretrained_embeddings = TEXT.vocab.vectors

    model.embedding.weight.data.copy_(pretrained_embeddings)
    
    UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

    model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
    model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)
    
    
    # Get w
    
    from collections import Counter
    class_distrbution = Counter()
    for text in y_kfold:
        class_distrbution[text] += 1


    class_weights = [class_distrbution[i] for i in LABELS.vocab.stoi.keys()]
    class_weights_normalized = [max(class_weights)/i for i in class_weights]


    w = torch.Tensor(class_weights_normalized)
    w = w.to(device)

    optimizer = optim.Adam(model.parameters())
#     criterion = nn.CrossEntropyLoss(weight=w)
    criterion = nn.CrossEntropyLoss()
        
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    criterion = criterion.to(device)

    train_iterator = data.Iterator(train_data, batch_size=batch_size, sort_key=lambda x: len(x.text), device=device)
    val_iterator = data.Iterator(val_data, batch_size=batch_size, sort_key=lambda x: len(x.text), device=device)

    for epoch in range(epochs):
        train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
        val_loss, val_acc = evaluate(model, val_iterator, criterion)
        print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
        print(f'| Epoch: {epoch+1:02} | Valid Loss: {val_loss:.3f} | Valid Acc: {val_acc*100:.2f}%')
        print('################################################################################')
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            print('Saving Model ...')
            torch.save(model.state_dict(), 'Best_Model_'+str(best_val_loss)[:5]+'.pt')
            print('*****************************************************')
            print('best record: [epoch %d], [val loss %.5f], [val acc %.5f]' % (epoch, val_loss, val_acc))
            print('*****************************************************')
    
    val_loss, val_acc = evaluate(model, val_iterator, criterion) 
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        print('Saving Model ...')
        torch.save(model.state_dict(), 'Best_Model_'+str(best_val_loss)[:5]+'.pt')
        print('*****************************************************')
        print('best record: [epoch %d], [val loss %.5f], [val acc %.5f]' % (epoch, val_loss, val_acc))
        print('*****************************************************')
    
#     if val_acc > best_valid_acc:
#         best_valid_acc = val_acc
#         print('Saving Model ...')
#         torch.save(model.state_dict(), 'Best_Model_'+str(val_acc)+'.pt')
#         print('*****************************************************')
#         print('best record: [epoch %d], [val loss %.5f], [val acc %.5f]' % (epoch, val_loss, val_acc))
#         print('*****************************************************')
        
    print(f'Val. Loss: {val_loss:.3f} | Val. Acc: {val_acc*100:.2f}% |')

    _history.append([val_loss, val_acc])
    fold_index += 1

_history = np.asarray(_history)
loss = np.mean(_history[:, 0])
acc = np.mean(_history[:, 1])

print('***** Cross Validation Result *****')
print(f'LOSS: {loss}, ACC: {acc}')

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
.vector_cache/glove.6B.zip: 0.00B [00:00, ?B/s]

***** Running Training *****
Now fold: 1 / 5


.vector_cache/glove.6B.zip: 862MB [06:26, 2.23MB/s]                              
100%|█████████▉| 399555/400000 [00:37<00:00, 10831.18it/s]

Embedding size: torch.Size([717, 300]).
| Epoch: 01 | Train Loss: 0.924 | Train Acc: 64.13%
| Epoch: 01 | Valid Loss: 0.616 | Valid Acc: 80.58%
################################################################################
| Epoch: 02 | Train Loss: 0.485 | Train Acc: 83.98%
| Epoch: 02 | Valid Loss: 0.416 | Valid Acc: 85.94%
################################################################################
| Epoch: 03 | Train Loss: 0.320 | Train Acc: 89.13%
| Epoch: 03 | Valid Loss: 0.372 | Valid Acc: 86.94%
################################################################################


100%|█████████▉| 399555/400000 [00:50<00:00, 10831.18it/s]

| Epoch: 04 | Train Loss: 0.210 | Train Acc: 94.34%
| Epoch: 04 | Valid Loss: 0.325 | Valid Acc: 89.51%
################################################################################
| Epoch: 05 | Train Loss: 0.139 | Train Acc: 97.46%
| Epoch: 05 | Valid Loss: 0.305 | Valid Acc: 89.51%
################################################################################
Val. Loss: 0.298 | Val. Acc: 89.84% |
***** Running Training *****
Now fold: 2 / 5
Embedding size: torch.Size([708, 300]).
| Epoch: 01 | Train Loss: 0.908 | Train Acc: 67.79%
| Epoch: 01 | Valid Loss: 0.560 | Valid Acc: 84.14%
################################################################################
| Epoch: 02 | Train Loss: 0.463 | Train Acc: 84.98%
| Epoch: 02 | Valid Loss: 0.396 | Valid Acc: 86.92%
################################################################################
| Epoch: 03 | Train Loss: 0.306 | Train Acc: 90.34%
| Epoch: 03 | Valid Loss: 0.356 | Valid Acc: 86.00%
#################################

In [ ]:
LOSS: 0.5449422478675843, ACC: 0.842294979095459
        
LOSS: 0.4586620181798935, ACC: 0.861218586564064

LOSS: 0.3850980430841446, ACC: 0.8524718940258026
        
LOSS: 0.3720459468662739, ACC: 0.8570932596921921

LOSS: 0.3683295398950577, ACC: 0.8664682626724243 # lower
        
LOSS: 0.3577434681355953, ACC: 0.8683201134204864 # spel checker contraction

LOSS: 0.35579548478126527, ACC: 0.8727182626724244


In [134]:
model.load_state_dict(torch.load('Best_Model_0.305.pt'))

val_loss, val_acc = evaluate(model, valid_iterator, criterion) 
val_loss

RuntimeError: Error(s) in loading state_dict for CNN:
	size mismatch for embedding.weight: copying a param with shape torch.Size([720, 300]) from checkpoint, the shape in current model is torch.Size([711, 300]).

In [169]:
# best record: [epoch 4], [val loss 0.60001], [val acc 0.83239]

Finally, let's run our model on the test set!

In [251]:
# model.load_state_dict(torch.load('tut5-model.pt'))

# test_loss, test_acc = evaluate(model, test_iterator, criterion)

# print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

In [480]:
# def predict_sentiment(model, tokenizer, sentence):
#     model.eval()
#     tokens = tokenizer.tokenize(sentence)
#     tokens = tokens#[:max_input_length-2]
#     indexed = [init_token_idx] + tokenizer.convert_tokens_to_ids(tokens) + [eos_token_idx]
#     tensor = torch.LongTensor(indexed).to(device)
#     tensor = tensor.unsqueeze(0)
#     prediction = torch.sigmoid(model(tensor))
#     return prediction

import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence, min_len = 4):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    preds = torch.softmax(model(tensor), dim=1)
#     preds = torch.sigmoid(model(tensor))
#     preds = model(tensor)
    return preds

In [481]:
print(LABELS.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7f354edce2f0>, {'Depression': 0, 'Alcohol': 1, 'Suicide': 2, 'Drugs': 3})


In [482]:
# predict_class(model, "Who is Keyser Söze?")

In [483]:
predict_sentiment(model, "weed addiction")

tensor([[0.0117, 0.2914, 0.0116, 0.6853]], device='cuda:0',
       grad_fn=<SoftmaxBackward>)

In [484]:
# predict(model, test_iterator, criterion)

In [485]:
test_data = pd.read_csv("./data/Test_process.csv")
# test_data = pd.read_csv("./data/Test.csv")

test_data.head()

,ID,text
0,02V56KMO,how to overcome bad feelings and emotions
1,03BMGTOK,feel like giving up in life
2,03LZVFM6,was so depressed feel like got no strength to ...
3,0EPULUM5,feel so low especially since had no one to tal...
4,0GM4C5GD,can be successful when am a drug addict


In [486]:
test_data.iloc[0]['text']

'how to overcome bad feelings and emotions'

In [487]:
len(test_data)

309

In [488]:
# Inference

# Load Best Model
# model.load_state_dict(torch.load('Best_Model_Bert_0.23.pt'))

y_pred = np.zeros((309, 4))
for i in range(len(test_data)):
    y_pred[i] = predict_sentiment(model, test_data.iloc[i]['text']).data.cpu()

In [489]:
y_pred.shape

(309, 4)

In [68]:
# y_pred

## Optimus

In [50]:
!pip install optuna

In [53]:
class load_data(object):
    def __init__(self, SEED=1234):
        torch.manual_seed(SEED)
        torch.cuda.manual_seed(SEED)
        torch.backends.cudnn.deterministic = True

        TEXT = data.Field(tokenize = 'spacy')
        LABELS = data.LabelField(dtype = torch.float)


#         self.train_data, self.test_data = datasets.IMDB.splits(TEXT, LABEL)
        
        self.train_data, self.test_data = data.TabularDataset.splits(
            path='./data', train='Train_process.csv',
            test='Test_process.csv', format='csv', skip_header=True,
            fields=[('ID', None),
                    ('text', TEXT),
                    ('label', LABELS)])

        self.SEED = SEED


    def get_fold_data(self, num_folds=10):
        """
        More details about 'fields' are available at 
        https://github.com/pytorch/text/blob/master/torchtext/datasets/imdb.py
        """

        TEXT = data.Field(tokenize='spacy')
        LABELS = data.LabelField(dtype=torch.float)
        fields = [('text', TEXT), ('label', LABELS)]
        
#         kf = KFold(n_splits=num_folds, shuffle=True, random_state=self.SEED)
        kf = StratifiedKFold(n_splits=num_folds, random_state=self.SEED)
        
        train_data_arr = np.array(self.train_data.examples)

#         ipdb.set_trace()
        for train_index, val_index in kf.split(train_data_arr, y_kfold):
            yield(
                TEXT,
                LABELS,
                data.Dataset(train_data_arr[train_index], fields=fields),
                data.Dataset(train_data_arr[val_index], fields=fields),
            )
    
    def get_test_data(self):
        return self.test_data

In [54]:
# best_loss_val = 0.8

def objective(trial):
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')    
    
    
    lr  = trial.suggest_loguniform('lr', 1e-4, 1e-2)
#     optim_ = trial.suggest_categorical('optim_',[optim.SGD, optim.RMSprop,optim.Adam])
    
    
    momentum = trial.suggest_uniform('momentum', 0.5, 0.9)
    
    
    
    data_generator = load_data()
    _history = []
    device = None
    model = None
    criterion = None
    fold_index = 0
    num_folds = 5
    batch_size = 32
    epochs = 20

    import torch.optim as optim

    # optimizer = optim.Adam(model.parameters())

    best_valid_acc = 0.9
    best_val_loss = 0.2

    for TEXT, LABELS, train_data, val_data in data_generator.get_fold_data(num_folds=num_folds):
#         print("***** Running Training *****")
#         print(f"Now fold: {fold_index + 1} / {num_folds}")

        TEXT.build_vocab(train_data,
                         vectors = "glove.6B.300d",
                         unk_init = torch.Tensor.normal_)

#         print(f'Embedding size: {TEXT.vocab.vectors.size()}.')
        LABELS.build_vocab(train_data) # For converting str into float labels.

    #     Model(len(TEXT.vocab), embedding_dim, hidden_dim,
    #         output_dim, num_layers, dropout, TEXT.vocab.vectors, embedding_trainable)

        INPUT_DIM = len(TEXT.vocab)
        EMBEDDING_DIM = 300
        N_FILTERS = 100
        FILTER_SIZES = [2,3,4]
        OUTPUT_DIM = len(LABELS.vocab)
        DROPOUT = 0.5
        PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

        model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

        pretrained_embeddings = TEXT.vocab.vectors

        model.embedding.weight.data.copy_(pretrained_embeddings)

        UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

        model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
        model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)


        # Get w
        from collections import Counter
        class_distrbution = Counter()
        for text in y_kfold:
            class_distrbution[text] += 1


        class_weights = [class_distrbution[i] for i in LABELS.vocab.stoi.keys()]
        class_weights_normalized = [max(class_weights)/i for i in class_weights]


        w = torch.Tensor(class_weights_normalized)
        w = w.to(device)

#         optimizer = optim.Adam(model.parameters())
#         criterion = nn.CrossEntropyLoss(weight=w)
        
        optimizer = optim.SGD(model.parameters(), lr = lr, momentum=momentum, 
                      weight_decay=0, dampening=0, nesterov=False)
        
#         optimizer = optim.Adam(model.parameters(), lr = 1e-4, momentum=0.9, 
#                       weight_decay=0, dampening=0, nesterov=True)
    
        criterion = nn.CrossEntropyLoss(weight = w)
        
        

        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model = model.to(device)
        criterion = criterion.to(device)

        train_iterator = data.Iterator(train_data, batch_size=batch_size, sort_key=lambda x: len(x.text), device=device)
        val_iterator = data.Iterator(val_data, batch_size=batch_size, sort_key=lambda x: len(x.text), device=device)

        for epoch in range(epochs):
            train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
            val_loss, val_acc = evaluate(model, val_iterator, criterion)
#             print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
#             print(f'| Epoch: {epoch+1:02} | Valid Loss: {val_loss:.3f} | Valid Acc: {val_acc*100:.2f}%')
#             print('################################################################################')
#             if val_loss < best_val_loss:
#                 best_val_loss = val_loss
#                 print('Saving Model ...')
#                 torch.save(model.state_dict(), 'Best_Model_'+str(best_val_loss)+'.pt')
#                 print('*****************************************************')
#                 print('best record: [epoch %d], [val loss %.5f], [val acc %.5f]' % (epoch, val_loss, val_acc))
#                 print('*****************************************************')

        val_loss, val_acc = evaluate(model, val_iterator, criterion) 

#         if val_loss < best_val_loss:
#                 best_val_loss = val_loss
#                 print('Saving Model ...')
#                 torch.save(model.state_dict(), 'Best_Model_'+str(best_val_loss)+'.pt')
#                 print('*****************************************************')
#                 print('best record: [epoch %d], [val loss %.5f], [val acc %.5f]' % (epoch, val_loss, val_acc))
#                 print('*****************************************************')

    #     if val_acc > best_valid_acc:
    #         best_valid_acc = val_acc
    #         print('Saving Model ...')
    #         torch.save(model.state_dict(), 'Best_Model_'+str(val_acc)+'.pt')
    #         print('*****************************************************')
    #         print('best record: [epoch %d], [val loss %.5f], [val acc %.5f]' % (epoch, val_loss, val_acc))
    #         print('*****************************************************')

#         print(f'Val. Loss: {val_loss:.3f} | Val. Acc: {val_acc*100:.2f}% |')

        _history.append([val_loss, val_acc])
        fold_index += 1

    _history = np.asarray(_history)
    loss = np.mean(_history[:, 0])
    acc = np.mean(_history[:, 1])

    print('***** Cross Validation Result *****')
    print(f'LOSS: {loss}, ACC: {acc}')
    
    # Handle pruning based on the intermediate value.
    if trial.should_prune():
        raise optuna.exceptions.TrialPruned()
            
            
    return loss

In [55]:
import optuna

sampler = optuna.samplers.TPESampler()
study = optuna.create_study(sampler=sampler, direction='minimize')
study.optimize(func=objective, n_trials=100)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning:

Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.



***** Cross Validation Result *****
LOSS: 1.327958458662033, ACC: 0.6552496731281281


[I 2020-07-03 20:37:05,156] Finished trial#0 with value: 1.327958458662033 with parameters: {'lr': 0.0003017745339013328, 'momentum': 0.6526010976015626}. Best is trial#0 with value: 1.327958458662033.
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning:

Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.



***** Cross Validation Result *****
LOSS: 0.9759525686502457, ACC: 0.8164930582046509


[I 2020-07-03 20:37:57,548] Finished trial#1 with value: 0.9759525686502457 with parameters: {'lr': 0.0010456463513600518, 'momentum': 0.822185665013464}. Best is trial#1 with value: 0.9759525686502457.
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning:

Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.



...

***** Cross Validation Result *****
LOSS: 0.49456333070993425, ACC: 0.8545469641685486


[I 2020-07-03 21:12:02,331] Finished trial#62 with value: 0.49456333070993425 with parameters: {'lr': 0.00474262609991237, 'momentum': 0.8863940479394523}. Best is trial#54 with value: 0.4926346495747566.
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning:

Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.



***** Cross Validation Result *****
LOSS: 0.49470569267868997, ACC: 0.8579613149166108


[I 2020-07-03 21:12:31,992] Finished trial#63 with value: 0.49470569267868997 with parameters: {'lr': 0.004486400306359615, 'momentum': 0.8998348902346536}. Best is trial#54 with value: 0.4926346495747566.
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning:

Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.



KeyboardInterrupt: 

In [ ]:
# import optuna

# sampler = optuna.samplers.TPESampler()
# study = optuna.create_study(sampler=sampler, direction='maximize')
# study.optimize(func=objective, n_trials=500)

# Submission

In [490]:
import pandas as pd

In [491]:
sample.tail()

,ID,Depression,Alcohol,Suicide,Drugs
304,Z9A6ACLK,0,0,0,0
305,ZDUOIGKN,0,0,0,0
306,ZHQ60CCH,0,0,0,0
307,ZVIJMA4O,0,0,0,0
308,ZYIFAY98,0,0,0,0


In [492]:
(test.ID == sample.ID).all()

True

In [493]:
LABELS.vocab.stoi.keys()

dict_keys(['Depression', 'Alcohol', 'Suicide', 'Drugs'])

In [494]:
predictions = pd.DataFrame(y_pred, columns=LABELS.vocab.stoi.keys())
predictions.head()

,Depression,Alcohol,Suicide,Drugs
0,0.766105,0.024983,0.197407,0.011505
1,0.970994,0.002207,0.023719,0.003081
2,0.992472,0.000565,0.006251,0.000712
3,0.982532,0.001679,0.013480,0.002308
4,0.558778,0.141814,0.108502,0.190906


In [495]:
result = pd.concat([sample[['ID']], predictions], axis=1)

In [496]:
result.head()

,ID,Depression,Alcohol,Suicide,Drugs
0,02V56KMO,0.766105,0.024983,0.197407,0.011505
1,03BMGTOK,0.970994,0.002207,0.023719,0.003081
2,03LZVFM6,0.992472,0.000565,0.006251,0.000712
3,0EPULUM5,0.982532,0.001679,0.013480,0.002308
4,0GM4C5GD,0.558778,0.141814,0.108502,0.190906


In [497]:
result.to_csv('./Pretrained_glove_300_text_contraction_all_0.28_softmax.csv', 
              index=False)

In [231]:
# np.sum([0.921355,0.010272,0.058423,0.009950]) # import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence, min_len = 5):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

1.0

In [ ]:
class load_data(object):
    def __init__(self, SEED=1234):
        torch.manual_seed(SEED)
        torch.cuda.manual_seed(SEED)
        torch.backends.cudnn.deterministic = True

        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        
        init_token = tokenizer.cls_token
        eos_token = tokenizer.sep_token
        pad_token = tokenizer.pad_token
        unk_token = tokenizer.unk_token
        
        init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
        eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
        pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
        unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)
        
        LABELS = data.LabelField(dtype = torch.float)

        TEXT = data.Field(batch_first = True,
                          use_vocab = False,
        #                   tokenize = tokenize_and_cut,
                          preprocessing = tokenizer.convert_tokens_to_ids,
                          init_token = init_token_idx,
                          eos_token = eos_token_idx,
                          pad_token = pad_token_idx,
                          unk_token = unk_token_idx)


#         self.train_data, self.test_data = datasets.IMDB.splits(TEXT, LABEL)
        
        self.train_data, self.test_data = data.TabularDataset.splits(
            path='./data', train='Train_process.csv',
            test='Test_process.csv', format='csv', skip_header=True,
            fields=[('ID', None),
                    ('text', TEXT),
                    ('label', LABELS)])

        self.SEED = SEED


    def get_fold_data(self, num_folds=10):
        """
        More details about 'fields' are available at 
        https://github.com/pytorch/text/blob/master/torchtext/datasets/imdb.py
        """

        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        
        init_token = tokenizer.cls_token
        eos_token = tokenizer.sep_token
        pad_token = tokenizer.pad_token
        unk_token = tokenizer.unk_token
        
        init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
        eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
        pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
        unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)
        
        LABELS = data.LabelField(dtype = torch.float)

        TEXT = data.Field(batch_first = True,
                          use_vocab = False,
        #                   tokenize = tokenize_and_cut,
                          preprocessing = tokenizer.convert_tokens_to_ids,
                          init_token = init_token_idx,
                          eos_token = eos_token_idx,
                          pad_token = pad_token_idx,
                          unk_token = unk_token_idx)

        fields = [('text', TEXT), ('label', LABELS)]
        
#         kf = KFold(n_splits=num_folds, shuffle=True, random_state=self.SEED)
        kf = StratifiedKFold(n_splits=num_folds, random_state=self.SEED)
        
        train_data_arr = np.array(self.train_data.examples)

#         ipdb.set_trace()
        for train_index, val_index in kf.split(train_data_arr, y_kfold):
            yield(
                TEXT,
                LABELS,
                data.Dataset(train_data_arr[train_index], fields=fields),
                data.Dataset(train_data_arr[val_index], fields=fields),
            )
    
    def get_test_data(self):
        return self.test_data

In [ ]:
# best_loss_val = 0.8

def objective(trial):
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')    
    
    
    lr  = trial.suggest_loguniform('lr', 1e-5, 1e-1)
#     optim_ = trial.suggest_categorical('optim_',[optim.SGD, optim.RMSprop,optim.Adam])
    
    
    momentum = trial.suggest_uniform('momentum', 0, 0.9)
    
    
    
#     data_generator = load_data()
#     _history = []
#     device = None
#     model = None
#     criterion = None
#     fold_index = 0
#     num_folds = 5
#     batch_size = 32
#     epochs = 20
    
    HIDDEN_DIM = 500
    OUTPUT_DIM = len(LABELS.vocab)
    N_LAYERS = 2
    BIDIRECTIONAL = True
    DROPOUT = 0.5

    import torch.optim as optim

    # optimizer = optim.Adam(model.parameters())

    best_valid_acc = 0.9
    best_val_loss = 0.2

    for TEXT, LABELS, train_data, val_data in data_generator.get_fold_data(num_folds=num_folds):
#         print("***** Running Training *****")
#         print(f"Now fold: {fold_index + 1} / {num_folds}")

        TEXT.build_vocab(train_data,
                         vectors = "glove.6B.300d",
                         unk_init = torch.Tensor.normal_)

#         print(f'Embedding size: {TEXT.vocab.vectors.size()}.')
        LABELS.build_vocab(train_data) # For converting str into float labels.

    #     Model(len(TEXT.vocab), embedding_dim, hidden_dim,
    #         output_dim, num_layers, dropout, TEXT.vocab.vectors, embedding_trainable)

        HIDDEN_DIM = 500
        OUTPUT_DIM = len(LABELS.vocab)
        N_LAYERS = 2
        BIDIRECTIONAL = True
        DROPOUT = 0.5

        model = BERTGRUSentiment(model,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)


        # Get w
        from collections import Counter
        class_distrbution = Counter()
        for text in y_kfold:
            class_distrbution[text] += 1


        class_weights = [class_distrbution[i] for i in LABELS.vocab.stoi.keys()]
        class_weights_normalized = [max(class_weights)/i for i in class_weights]


        w = torch.Tensor(class_weights_normalized)
        w = w.to(device)

#         optimizer = optim.Adam(model.parameters())
#         criterion = nn.CrossEntropyLoss(weight=w)
        
        optimizer = optim.SGD(model.parameters(), lr = lr, momentum=momentum, 
                      weight_decay=0, dampening=0, nesterov=False)
        
#         optimizer = optim.Adam(model.parameters(), lr = 1e-4, betas=(0.9, 0.999), 
#                       weight_decay=0.0, amsgrad=False)
    
        criterion = nn.CrossEntropyLoss(weight = w)
        
        

        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model = model.to(device)
        criterion = criterion.to(device)

        train_iterator = data.Iterator(train_data, batch_size=batch_size, sort_key=lambda x: len(x.text), device=device)
        val_iterator = data.Iterator(val_data, batch_size=batch_size, sort_key=lambda x: len(x.text), device=device)

        for epoch in range(epochs):
            train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
            val_loss, val_acc = evaluate(model, val_iterator, criterion)
#             print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
#             print(f'| Epoch: {epoch+1:02} | Valid Loss: {val_loss:.3f} | Valid Acc: {val_acc*100:.2f}%')
#             print('################################################################################')
#             if val_loss < best_val_loss:
#                 best_val_loss = val_loss
#                 print('Saving Model ...')
#                 torch.save(model.state_dict(), 'Best_Model_'+str(best_val_loss)+'.pt')
#                 print('*****************************************************')
#                 print('best record: [epoch %d], [val loss %.5f], [val acc %.5f]' % (epoch, val_loss, val_acc))
#                 print('*****************************************************')

        val_loss, val_acc = evaluate(model, val_iterator, criterion) 

#         if val_loss < best_val_loss:
#                 best_val_loss = val_loss
#                 print('Saving Model ...')
#                 torch.save(model.state_dict(), 'Best_Model_'+str(best_val_loss)+'.pt')
#                 print('*****************************************************')
#                 print('best record: [epoch %d], [val loss %.5f], [val acc %.5f]' % (epoch, val_loss, val_acc))
#                 print('*****************************************************')

    #     if val_acc > best_valid_acc:
    #         best_valid_acc = val_acc
    #         print('Saving Model ...')
    #         torch.save(model.state_dict(), 'Best_Model_'+str(val_acc)+'.pt')
    #         print('*****************************************************')
    #         print('best record: [epoch %d], [val loss %.5f], [val acc %.5f]' % (epoch, val_loss, val_acc))
    #         print('*****************************************************')

#         print(f'Val. Loss: {val_loss:.3f} | Val. Acc: {val_acc*100:.2f}% |')

        _history.append([val_loss, val_acc])
        fold_index += 1

    _history = np.asarray(_history)
    loss = np.mean(_history[:, 0])
    acc = np.mean(_history[:, 1])

    print('***** Cross Validation Result *****')
    print(f'LOSS: {loss}, ACC: {acc}')
    
    # Handle pruning based on the intermediate value.
    if trial.should_prune():
        raise optuna.exceptions.TrialPruned()
            
            
    return loss